In [18]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads
from shapely.geometry import Point
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
### fire ignition locations
fire_ignition_df = pd.DataFrame({
    'point_id': [1, 2, 3],
    'x': [-2296098.82090622, -2294638.88375515, -2294543.37384807],
    'y': [1977182.58128173, 1977578.26518249, 1978765.31688476]
})

fire_ignition_gdf = gpd.GeoDataFrame(fire_ignition_df, crs=5070, geometry=[Point(xy) for xy in zip(fire_ignition_df.x, fire_ignition_df.y)]).to_crs(4326)
fire_ignition_gdf.to_csv('fire_frontiers/ignition_points.csv', index=False)

In [47]:
### Read shapefiles of fire frontier based on Paola's simulations
### Keep only the largest polygon
for point_id in [1, 2, 3]:
    fire_gdf = gpd.read_file('Shapefile_Simulation_Paola/point{}_10Hr.shp'.format(point_id))
    fire_gdf = fire_gdf[fire_gdf['Fire_Type']=='Expanding Fire'].reset_index(drop=True).to_crs(4326)
    # p1_gdf['area'] = p1_gdf.area
    ignition_buffer = fire_ignition_gdf.to_crs(5070).loc[fire_ignition_gdf['point_id']==point_id, 'geometry'].buffer(1).to_crs(4326).iloc[0]
    fire_gdf.loc[fire_gdf.shape[0]] = [None, 'ignition_buffer', None, None, 900, None, None, ignition_buffer]
    fire_gdf['geometry'] = fire_gdf.to_crs(5070).simplify(1).to_crs(4326)
    fire_gdf.to_csv('fire_frontiers/point{}_10Hr_simplified.csv'.format(point_id), index=False)